# Thống kê

In [17]:
# đọc file data
import pandas as pd
df=pd.read_csv('data_10000.csv')
df = df.drop("link", axis=1)
df.head()


,title,author,language,avg_ratings,ratings_count,reviews_count,publisher,publish_year,num_pages,page_format,genres
0,The Hunger Games,Suzanne Collins,English,4.33,"7,971,601","196,977",Scholastic Press,2008.0,374.0,Hardcover,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas..."
1,Harry Potter and the Order of the Phoenix,J.K. Rowling,English,4.50,"3,150,618","57,202",Scholastic Inc.,2004.0,912.0,Paperback,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',..."
2,Pride and Prejudice,Jane Austen,English,4.28,"3,948,185","100,606",Modern Library,2000.0,279.0,Paperback,"['Classics', 'Fiction', 'Romance', 'Historical..."
3,To Kill a Mockingbird,Harper Lee,English,4.27,"5,696,477","110,338",Harper Perennial Modern Classics,2006.0,323.0,Paperback,"['Classics', 'Fiction', 'Historical Fiction', ..."
4,The Book Thief,Markus Zusak,English,4.39,"2,367,106","136,148",Alfred A. Knopf,2006.0,552.0,Hardcover,"['Historical Fiction', 'Fiction', 'Young Adult..."


In [22]:
# mô tả các thống kê tổng quan về tập dữ liệu
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          9999 non-null   object 
 1   author         9999 non-null   object 
 2   language       9788 non-null   object 
 3   avg_ratings    9999 non-null   float64
 4   ratings_count  9999 non-null   object 
 5   reviews_count  9999 non-null   object 
 6   publisher      9627 non-null   object 
 7   publish_year   9938 non-null   float64
 8   num_pages      9875 non-null   float64
 9   page_format    9955 non-null   object 
 10  genres         9999 non-null   object 
dtypes: float64(3), object(8)
memory usage: 859.4+ KB
None


## Thống kê mô tả

In [27]:
# thống kê mô tả trực quan về các đặc trưng
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
avg_ratings,9999.0,4.056113,0.306140,0.0,3.89,4.07,4.24,5.0
publish_year,9938.0,2006.714933,88.727561,1.0,2003.00,2008.00,2013.00,9999.0
num_pages,9875.0,384.260354,290.222828,0.0,257.00,343.00,442.00,8258.0
